In [1]:
import pandas as pd
import stumpy
import numpy as np
import datetime as dt
import random
import math
import pickle
import sys

from statistics import mean
from tqdm.auto import tqdm
from multiprocessing import Pool

import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
## PART 3
shapelet_samples_list = range(0, 18)
shapelet_size_list = [500]
clf_samples_list = [400000]

parameter_list = np.array(np.meshgrid(shapelet_samples_list, shapelet_size_list, clf_samples_list)).T.reshape(-1,3)

print(parameter_list)



[[     0    500 400000]
 [     1    500 400000]
 [     2    500 400000]
 [     3    500 400000]
 [     4    500 400000]
 [     5    500 400000]
 [     6    500 400000]
 [     7    500 400000]
 [     8    500 400000]
 [     9    500 400000]
 [    10    500 400000]
 [    11    500 400000]
 [    12    500 400000]
 [    13    500 400000]
 [    14    500 400000]
 [    15    500 400000]
 [    16    500 400000]
 [    17    500 400000]]


In [25]:
from sklearn.model_selection import StratifiedKFold
'''
Perform cross-validation of sklearn classifier on training data samples 

Input:
   
    clf: sklearn classifier object
    X: x values
    y: y values
    topk: k values for evaluation metrics
    n_splits: Number of folds.
Output:
    list scores, one for each fold, where each score is of length topk with accuracy for validate data
'''

def cross_validate(clf,X,y, topk=[1,3,5],n_splits=5):
    
    #xx = X[0,:30]
    #print(xx)
    #yy = y[:30]
    #print(yy)
    skf = StratifiedKFold(n_splits=n_splits)

    #yyy = []
    cv_score_list = []    
    for train_index, validate_index in skf.split(X,y):
        print("\n\n\n=================\n","train_index",train_index)
        print("\n\n\n=================\n","validate_index",validate_index)

        X_train, X_validate = X[train_index], X[validate_index]
        #y_train, y_validate = y[train_index], y[validate_index]
        
        
        
        y_train = [y[j] for j in train_index]
        #print(X[train_index].shape,len(y_train))
        
        y_validate = [y[j] for j in validate_index]
        print("\ny_validate:",y_validate)
        
        #print("\n")

        clf.fit(X_train,y_train)
        y_prob = clf.predict_proba(X_validate)

        #print(y_prob.shape)


        cv_scores = []
        for k in topk:
            correct = 0
            for i in range(len(y_prob)):
                ind = np.argpartition(y_prob[i], -k)[-k:]
                if y_validate[i] in ind:
                    correct += 1
            #print(correct/len(y_prob))
            cv_scores.append(correct/len(y_prob))

        cv_score_list.append(cv_scores)

    return cv_score_list
    
    
    
    
    

In [26]:

'''
Evaluate performance of sklearn classifier on data samples - 90/10 training testing split

Input:
    
    clf: sklearn classifier object
    X: x values
    y: y values
    topk: k values for evaluation metrics
    bCrossValidate: A boolean variable defining if cross-validation is required
    n_splits: Number of cross-validation folds
Output:
    list of length topk with accuracy for testing data
    list scores, one for each fold, where each score is of length topk with accuracy for validate data, return -1 if bCrossValidate = False
'''

def classifier_performance( clf, X, y, topk=[1,3,5],bCrossValidate=True, n_splits=5):
    
    #print(type(X),type(y))
    #print(X.shape,len(y))
    cv_score_list = -1
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    #print("X_train.shape",X_train.shape,"X_test.shape",X_test.shape,"y_train",len(y_train),"y_test.shape",len(y_test))
    
    if bCrossValidate == True:
        cv_score_list = cross_validate(clf,X_train,y_train,topk,n_splits)
        #print("cv_score_list", cv_score_list)
    
    #print("\n******************\ncross validation ends here\n\n\n")
    clf.fit(X_train, y_train)
    y_prob = clf.predict_proba(X_test)
    
    #print(y_prob.shape)
    
    scores = []
    for k in topk:
        correct = 0
        for i in range(len(y_prob)):
            ind = np.argpartition(y_prob[i], -k)[-k:]
            if y_test[i] in ind:
                correct += 1
        scores.append(correct/len(y_prob))
    
    return cv_score_list, scores

In [27]:
import os
for parameters in parameter_list:
    
    
    filename = '../results/data/X/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1]) + 'samples=' + str(parameters[2])
    if os.path.isfile(filename) == False:
        continue
    print(filename)
    with open(filename, 'rb') as f:
        X = pickle.load(f)
    
    
    filename = '../results/data/y/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1]) + 'samples=' + str(parameters[2])
    if os.path.isfile(filename) == False:
        continue
    print(filename)
    with open(filename, 'rb') as f:
        y = pickle.load(f)
    
    bCrossValidate = True
    Num_Instance = 1000
    n_splits = 5
    
    clf = RandomForestClassifier()
    cv_score_list, scores = classifier_performance(clf, X[:Num_Instance,:], y[:Num_Instance],bCrossValidate=bCrossValidate)
    
    # To run on entire dataset, replace the above line with the following
    #cv_score_list, scores = classifier_performance(clf, X, y,bCrossValidate=bCrossValidate,n_splits=n_splits)
    
    print("",cv_score_list,"\n",scores)
    print("===========================================")
    
    
    #outfile_name = "../results/scores/" + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1]) + 'samples=' + str(parameters[2])
    
    #with open(outfile_name, 'wb') as f:
    #    pickle.dump(scores, f)

../results/data/X/num=4size=500samples=400000
../results/data/y/num=4size=500samples=400000



 train_index [ 40  56  59  63  69  74  79  80  82  88  89  91  93  98 100 101 103 117
 118 121 122 123 126 127 129 134 140 141 142 146 147 148 150 155 157 159
 161 162 163 164 167 168 170 172 175 176 182 188 189 190 192 194 195 197
 198 199 201 206 207 208 209 213 214 215 216 217 218 220 222 223 226 227
 228 230 233 234 236 238 239 241 242 243 244 245 247 248 249 250 251 252
 253 255 256 257 259 260 261 262 264 267 268 270 272 273 274 275 276 277
 278 279 280 281 283 284 285 287 288 289 290 291 292 293 294 295 296 297
 299 300 301 302 303 304 306 307 308 309 310 311 312 313 314 315 316 317
 319 320 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337
 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355
 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373
 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391
 393 394 395 396

C:\Users\Uzma Maroof\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(





 train_index [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  57  58  60  61  62  64  65  66  67  68  69  70  71  72  73  75  76
  77  78  81  83  84  85  86  87  90  92  94  95  96  97  99 102 103 104
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 124 125
 128 130 131 132 133 135 136 137 138 139 141 143 144 145 149 151 152 153
 154 156 157 158 160 162 165 166 169 171 173 174 177 178 179 180 181 182
 183 184 185 186 187 188 190 191 193 196 200 202 203 204 205 210 211 212
 214 216 219 221 224 225 229 231 232 235 237 240 241 244 246 247 249 250
 254 258 261 263 265 266 269 271 273 277 278 282 283 284 286 287 291 292
 293 294 297 298 304 305 310 311 312 313 316 318 319 321 324 328 330 332
 333 334 336 337 339 340 344 347 348 351 352 354 356 359 365 367 370 372
 373 374 375 376 377 378 381 383 38




 train_index [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 104 105 106 107 108
 109 110 111 112 113 114 115 116 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 24

 [[0.016666666666666666, 0.05555555555555555, 0.06666666666666667], [0.011111111111111112, 0.03888888888888889, 0.06111111111111111], [0.0, 0.03888888888888889, 0.06111111111111111], [0.011111111111111112, 0.03888888888888889, 0.05555555555555555], [0.022222222222222223, 0.06666666666666667, 0.08333333333333333]] 
 [0.03, 0.04, 0.05]
../results/data/X/num=6size=500samples=400000
../results/data/y/num=6size=500samples=400000


C:\Users\Uzma Maroof\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(





 train_index [ 41  66  67  74  75  81  82  83  97 102 106 109 111 113 115 119 120 123
 124 127 129 131 136 137 143 144 148 149 151 152 154 155 158 162 163 165
 166 167 169 170 172 173 175 176 181 182 183 184 185 187 188 189 190 191
 193 195 196 197 201 202 203 205 206 208 212 213 214 215 216 217 219 220
 228 229 231 232 233 234 235 236 239 240 241 243 244 245 246 247 248 249
 250 252 254 259 260 261 262 264 265 266 267 268 269 270 271 272 273 274
 275 276 277 279 280 281 282 283 284 285 287 289 290 291 292 293 294 295
 296 297 298 299 300 301 302 303 304 305 306 307 309 310 311 312 313 314
 315 317 318 319 320 321 323 324 325 326 327 328 329 330 331 332 333 334
 335 336 337 338 339 340 342 343 344 345 346 347 348 349 350 351 352 353
 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371
 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389
 390 391 392 393 394 395 396 397 398 399 400 401 402 404 405 406 407 408
 409 410 411 412 413 414 415 416 41




 train_index [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 138 139 140 141 142 143 144
 145 146 147 150 152 153 154 155 156 157 158 159 160 161 162 163 164 165
 166 167 168 169 170 171 172 173 174 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 197 198 199 200 201 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 234 235 236 237 238 239 240 241 242 243
 246 248 249 250 251 252 253 254 25




 train_index [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 24